In [ ]:
import numpy as np
import pandas
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from matplotlib import colors

import griottes


# Read files with cell types

We doanload the single cell data provided by manual analysis.

In [ ]:
import urllib3
import shutil

url = 'https://github.com/BaroudLab/Griottes/releases/download/v1.0-alpha/zebrafish_cell_properties.csv'
filename = 'zebrafish_cell_properties.csv'

c = urllib3.PoolManager()

with c.request('GET',url, preload_content=False) as resp, open(filename, 'wb') as out_file:
    shutil.copyfileobj(resp, out_file)

resp.release_conn()

In [ ]:
zebrafish_cell_properties = pandas.read_csv('zebrafish_cell_properties.csv')

For illustration purposes, we assign different colors to the different cell types:

In [ ]:
n_colors = len(zebrafish_cell_properties.cell_type.unique())
color_list = [plt.cm.Set3(i) for i in range(n_colors)]
colors = [color_list[zebrafish_cell_properties.loc[i, 'cell_type']] for i in range(len(zebrafish_cell_properties))]

zebrafish_cell_properties['color'] = colors
zebrafish_cell_properties['legend'] = zebrafish_cell_properties['cell_properties']

# Create Delaunay network

We have prepared `new_prop`, the dataframe containing the relevant single-cell data information. It is provided as an input to `griottes` which then generates a Delaunay network. 

In [ ]:
descriptors = ['label', 'cell_type', 'color', 'legend', 'x', 'y', 'z','cell_properties']

In [ ]:
G = griottes.generate_delaunay_graph(zebrafish_cell_properties, 
                                                 descriptors = descriptors, 
                                                 distance=35,
                                                 )

The generated network can be visualised using specific network visualisation tools:

In [ ]:
griottes.plot_2D(G,
                alpha_line = 0.6,
                scatterpoint_size = 8,
                legend = True,
                legend_fontsize = 14,
                figsize = (7,7))

plt.tight_layout()